# Analysis of the Hexapod

This notebook is to analyze the hexapod behaviour and do the debug, especially for the [FRACAS-110](https://jira.lsstcorp.org/browse/FRACAS-110) and [FRACAS-111](https://jira.lsstcorp.org/browse/FRACAS-111).

## Import Modules

This notebook needs to setup the **ts_aos_utils** under the **notebooks/.user_setups**.

In [ ]:
%matplotlib inline
%matplotlib widget
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
from astropy.time import Time

from lsst.ts.aos.utils import DiagnosticsHexapod, EfdName

## Declaration of User-defined Functions

In [ ]:
async def show_all_data(
    diagnostics_hexapod: DiagnosticsHexapod,
    time_start: Time,
    time_end: Time,
    bits_status_word: list[int] | None = None,
    bits_application_status: list[int] | None = None,
) -> None:
    """
    Show all the data.

    Parameters
    ----------
    diagnostics_hexapod : `lsst.ts.aos.utils.DiagnosticsHexapod`
        Hexapod diagnostics instance.
    time_start : `astropy.time.core.Time`
        Start time.
    time_end : `astropy.time.core.Time`
        End time.
    bits_status_word : `list` or None, optional
        Interested bits of the status word (0x6041). (the default is None)
    bits_application_status : `list` or None, optional
        Interested bits of the application status. (the default is None)
    """

    # Query data
    data_application, time_operation_application = await diagnostics_hexapod.get_data_application(
        time_start, time_end)
    data_actuators, time_operation_actuators = await diagnostics_hexapod.get_data_actuators(
        time_start, time_end)
    data_electrical, time_operation_electrical = await diagnostics_hexapod.get_data_electrical(
        time_start, time_end)

    data_controller_state = await diagnostics_hexapod.get_data_controller_state(
        time_start, time_end)

    # Collect the bitmap
    status_word = np.array(
        [getattr(data_electrical, f"copleyStatusWordDrive{idx}")
         for idx in range(diagnostics_hexapod.NUM_ACTUATOR)]
    )
    bitmap_status_word = diagnostics_hexapod.map_bit_value(status_word, 16)

    # Draw the diagrams
    diagnostics_hexapod.plot_hexapod_position(data_application, time_operation_application)
    diagnostics_hexapod.plot_hexapod_position_error(data_application, time_operation_application)

    diagnostics_hexapod.plot_actuator_position(data_actuators, time_operation_actuators)
    diagnostics_hexapod.plot_actuator_position_error(data_actuators, time_operation_actuators)
    diagnostics_hexapod.plot_actuator_position_velocity(data_actuators, time_operation_actuators)

    diagnostics_hexapod.plot_actuator_current(data_electrical, time_operation_electrical)
    diagnostics_hexapod.plot_bus_voltage(data_electrical, time_operation_electrical)
    
    if bits_status_word is not None:
        for bit in bits_status_word:
            diagnostics_hexapod.plot_actuator_bit(
                time_operation_electrical,
                bitmap_status_word, bit,
                f"Status Word (bit {bit})",
            )

    if bits_application_status is not None:
        for bit in bits_application_status:
            diagnostics_hexapod.plot_application_status(
                data_controller_state,
                bit,
                timestamp_origin=data_electrical.private_sndStamp[0],
                title=f"Application Status (bit {bit})"
            )

## Instantiate the DiagnosticsHexapod Class

Notice that the UTC time is used when doing the query.

In [ ]:
diagnostics_hexapod = DiagnosticsHexapod(is_camera_hexapod=True, efd_name=EfdName.Usdf)

## Check the Data

In [ ]:
time_start = Time("2023-09-06T21:47:00", scale="utc", format="isot")
time_end = Time("2023-09-06T21:55:40", scale="utc", format="isot")

In [ ]:
await show_all_data(
    diagnostics_hexapod,
    time_start, time_end,
    bits_status_word=[7, 8, 11, 13],
    bits_application_status=[0, 13],
)

In [ ]:
plt.close("all")